In [ ]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error
from imblearn.over_sampling import SMOTE
import joblib

### The Student Performance Data Set
[The Student Performance](https://archive.ics.uci.edu/dataset/320/student+performance) Data Set is a collection of data gathered from secondary school students in Portugal. It was compiled to analyse the factors that influence academic success, particularly in mathematics and Portuguese language courses. The data was collected from two schools and includes a wide range of attributes related to student demographics, social and economic factors, and academic records.

### Load the data

In [3]:
# fetch dataset
student_performance = fetch_ucirepo(id=320)
# get df
student_performance_df = student_performance.data.original

### Explore and define the target data

In [ ]:
# variable information
student_performance.variables

In [4]:
# Keep only selected columns
selected_columns = ['G1', 'studytime', 'famsup', 'G3']
student_performance_df = student_performance_df[selected_columns]
# features and target
target = 'G3'
features = [col for col in student_performance_df.columns if col != target]

In [5]:
# Handle categorical variables via one-hot encoding
data_prepared = pd.get_dummies(student_performance_df[features])
X = data_prepared
y = student_performance_df[target]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=200)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
rmse

1.9739541289739775

## Save model and version
Create a function that saves the model , performance score and metadata everytime there is a change

In [6]:
import os
import joblib
import json
import datetime

def save_model_and_metadata(model, metrics, version, repo_name="MLops_demo", base_dir="/content/drive/MyDrive/01.RWML"):
    """
    Save model and metadata into the specified directory structure under the given base directory.

    Args:
        model: Trained machine learning model (e.g., RandomForestRegressor).
        metrics: Dictionary containing evaluation metrics (e.g., RMSE, accuracy).
        version: Model version (e.g., "v1", "v2").
        repo_name: Name of the repository (e.g., "MLops_demo").
        base_dir: Base directory where the repository resides.
    """
    # Construct the full repository path
    repo_dir = os.path.join(base_dir, repo_name)

    # Create paths for saving
    model_dir = os.path.join(repo_dir, version)
    os.makedirs(model_dir, exist_ok=True)

    # Save model
    model_path = os.path.join(model_dir, f"model_{version}.pkl")
    joblib.dump(model, model_path)

    # Save metadata
    metadata = {
        "model_version": version,
        "metrics": metrics,
        "timestamp": str(datetime.datetime.now())
    }
    metadata_path = os.path.join(model_dir, f"metadata_{version}.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

    print(f"Model saved to {model_path}")
    print(f"Metadata saved to {metadata_path}")


In [7]:
metrics = {"rmse": rmse}  #
save_model_and_metadata(rf_model, metrics, version="v2")

Model saved to /content/drive/MyDrive/01.RWML/MLops_demo/v2/model_v2.pkl
Metadata saved to /content/drive/MyDrive/01.RWML/MLops_demo/v2/metadata_v2.json


In [8]:
!cp '/content/drive/MyDrive/Colab Notebooks/MLops.ipynb' /content/drive/MyDrive/01.RWML/MLops_demo/v2/